In [1]:
import torch
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch import nn
from typing import List

In [3]:
x = torch.randint(1, 4, ( 2, 10, ))
x

tensor([[2, 2, 3, 2, 3, 3, 3, 1, 3, 2],
        [1, 1, 1, 2, 3, 3, 3, 2, 1, 2]])

In [9]:
size= 10
hours = torch.randint(1, 25, (size,)).sort()[0]
sixhours = hours // 6
day = torch.randint(1, 8, (1, )).expand_as(hours)
weekend = (day > 5).to(int)

In [11]:
day

tensor([7, 7, 7, 7, 7, 7, 7, 7, 7, 7])